# **Reranker RAG**

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereRerank
import warnings
warnings.filterwarnings('ignore')

loader = WebBaseLoader("https://python.langchain.com/docs/integrations/retrievers/cohere-reranker/")
docs = loader.load()

spiltter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=15)
chunks = spiltter.split_documents(docs)

In [15]:
import os
os.environ["COHERE_API_KEY"] = "API-KEY"

In [16]:
embadding = CohereEmbeddings(model="embed-english-v3.0" , user_agent='langchain')

In [7]:
vectorstore = FAISS.from_documents(chunks, embadding).as_retriever(search_kwargs={"k": 20})

In [8]:
query = "What is Cohere Reranking"
vectorstore.invoke(query)

[Document(id='cd186178-90d7-4dc3-8410-8b1b7d7a234a', metadata={'source': 'https://python.langchain.com/docs/integrations/retrievers/cohere-reranker/', 'title': 'Cohere reranker | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'Cohere is a Canadian startup that provides natural language processing models that help companies improve human-machine interactions.', 'language': 'en'}, page_content="This notebook shows how to use Cohere's rerank endpoint in a retriever. This builds on top of ideas in the ContextualCompressionRetriever.\n%pip install --upgrade --quiet  cohere\n%pip install --upgrade --quiet  faiss# OR  (depending on Python version)%pip install --upgrade --quiet  faiss-cpu"),
 Document(id='bca2f79f-72d7-4eab-8775-f18a9b1299a5', metadata={'source': 'https://python.langchain.com/docs/integrations/retrievers/cohere-reranker/', 'title': 'Cohere reranker | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'Cohere is a Canadian startup that provides natural language processing model

In [9]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [10]:
docs = vectorstore.invoke(query)
pretty_print_docs(docs)

Document 1:

This notebook shows how to use Cohere's rerank endpoint in a retriever. This builds on top of ideas in the ContextualCompressionRetriever.
%pip install --upgrade --quiet  cohere
%pip install --upgrade --quiet  faiss# OR  (depending on Python version)%pip install --upgrade --quiet  faiss-cpu
----------------------------------------------------------------------------------------------------
Document 2:

Doing reranking with CohereRerank‚Äã
Now let's wrap our base retriever with a ContextualCompressionRetriever. We'll add an CohereRerank, uses the Cohere rerank endpoint to rerank the returned results.
Do note that it is mandatory to specify the model name in CohereRerank!
----------------------------------------------------------------------------------------------------
Document 3:

Cohere is a Canadian startup that provides natural language processing models that help companies improve human-machine interactions.
------------------------------------------------------------

In [11]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

In [17]:
llm = Cohere(temperature=0)
compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vectorstore
)

In [18]:
compressed_docs = compression_retriever.invoke(
    "How to use Cohere Reranking"
)
pretty_print_docs(compressed_docs)

Document 1:

Doing reranking with CohereRerank‚Äã
Now let's wrap our base retriever with a ContextualCompressionRetriever. We'll add an CohereRerank, uses the Cohere rerank endpoint to rerank the returned results.
Do note that it is mandatory to specify the model name in CohereRerank!
----------------------------------------------------------------------------------------------------
Document 2:

This notebook shows how to use Cohere's rerank endpoint in a retriever. This builds on top of ideas in the ContextualCompressionRetriever.
%pip install --upgrade --quiet  cohere
%pip install --upgrade --quiet  faiss# OR  (depending on Python version)%pip install --upgrade --quiet  faiss-cpu
----------------------------------------------------------------------------------------------------
Document 3:

CohereRerank | Cohere


In [19]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm=Cohere(temperature=0), retriever=compression_retriever
)

In [21]:
chain({"query": "How to use Cohere Reranking"})

{'query': 'How to use Cohere Reranking',
 'result': " The provided text mentions using `CohereRerank` to rerank the returned results from a search. To use `CohereRerank`, one has to specify the model name, which is a mandatory field. You can install the required libraries (`cohere` and `faiss` or `faiss-cpu`) to utilize the Cohere retriever and rerank endpoint. \nUnfortunately, I don't have access to the internet here, and I cannot install the above-mentioned libraries. Feel free to install them to explore and experiment with CohereRerank and the rerank endpoint. \nIf you have any specific questions about installing or using CohereRerank, please let me know, and I will gladly assist you based on the available resources and my knowledge. "}